In [1]:
import pandas as pd
import numpy as np

# Load Daymet data
df = pd.read_csv("../basin_mean_forcing/daymet/spatial_avg_Guerneville.csv", parse_dates=["time"])
df.set_index("time", inplace=True)

# ---- Basic
p_mean = df["prcp"].resample("Y").sum().mean()
monthly = df["prcp"].resample("M").sum()
p_seasonality = monthly.groupby(monthly.index.month).mean().std() / monthly.groupby(monthly.index.month).mean().mean()

# ---- Snow fraction: tmax < 0C days with precip
snow_days = df[(df["tmax"] < 0) & (df["prcp"] > 1)].shape[0]
wet_days = df[df["prcp"] > 1].shape[0]
frac_snow = snow_days / wet_days if wet_days > 0 else 0

# ---- PET estimation using Hargreaves (simple method)
df["tavg"] = (df["tmax"] + df["tmin"]) / 2
df["pet"] = 0.0023 * (df["tmax"] - df["tmin"]).pow(0.5) * (df["tavg"] + 17.8) * 0.408
pet_mean = df["pet"].resample("Y").sum().mean()
aridity = pet_mean / p_mean

# ---- High precip events
high_thresh = df["prcp"].quantile(0.9)
df["high_prec"] = df["prcp"] > high_thresh
df["doy"] = df.index.dayofyear
df["high_group"] = (df["high_prec"] != df["high_prec"].shift()).cumsum()
high_events = df[df["high_prec"]].groupby("high_group")
high_freq = high_events.ngroups / (df.index.year.max() - df.index.year.min() + 1)
high_dur = high_events.size().mean()
high_timing = high_events["doy"].mean().mean()

# ---- Low precip events
df["low_prec"] = df["prcp"] < 1
df["low_group"] = (df["low_prec"] != df["low_prec"].shift()).cumsum()
low_events = df[df["low_prec"]].groupby("low_group")
low_freq = low_events.ngroups / (df.index.year.max() - df.index.year.min() + 1)
low_dur = low_events.size().mean()
low_timing = low_events["doy"].mean().mean()

# ---- Output
row = {
    "gauge_id": 11467000,
    "p_mean": p_mean,
    "pet_mean": pet_mean,
    "p_seasonality": p_seasonality,
    "frac_snow": frac_snow,
    "aridity": aridity,
    "high_prec_freq": high_freq,
    "high_prec_dur": high_dur,
    "high_prec_timing": high_timing,
    "low_prec_freq": low_freq,
    "low_prec_dur": low_dur,
    "low_prec_timing": low_timing
}

df_out = pd.DataFrame([row])
df_out.to_csv("camels_clim_11467000.txt", sep=";", index=False)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/wh/2ltd3_yj3gx4x07y15x8vhbw0000gn/T/ipykernel_44609/510614703.py:9: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  p_mean = df["prcp"].resample("Y").sum().mean()
/var/folders/wh/2ltd3_yj3gx4x07y15x8vhbw0000gn/T/ipykernel_44609/510614703.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df["prcp"].resample("M").sum()
/var/folders/wh/2ltd3_yj3gx4x07y15x8vhbw0000gn/T/ipykernel_44609/510614703.py:21: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  pet_mean = df["pet"].resample("Y").sum().mean()
